---
# Fair coin

Q: You toss a fair coin 400 times. What’s the probability that you get at least 220 heads?

A: We are dealing with a binomial distribution: $n = 400$, $k = 1\ldots220$, $p = 0.5$
\begin{equation}
f(k, n, p) = {{n}\choose{k}} p^k (1-p)^{(n-k)} = \left(\frac{n!}{k! (n-k)!}\right) p^k (1-p)^{(n-k)}
\end{equation}
$$\mu = n p$$
$$\sigma^2 = n p (1-p)$$

### Approximation

As $n$ gets large, the binomial distribution approaches the normal distribution. We have
$$\mu = n p = 400 \times 0.5 = 200$$
$$\sigma^2 = n p (1-p) = 400 \times 0.5 \times 0.5 = 100$$
$$\sigma = \sqrt{100} = 10$$
$$k - mean = 220 - 200 = 20 = 2 \times \sigma$$
from standard normal c.d.f. table, we see that $2 \sigma - \mu \approx 0.47725$, by symmetry
$$P(X>=220) = 1 - 0.5 - 0.47725 = 0.02275$$

### Exact answer

Factorials get untractably large with not-so-large $n$ (smaller than 400)
therefore we need an indirect (yet correct) strategy to compute the binomial
distribution. One such way is multiplicative:
\begin{equation}
{{n}\choose{k}} = \prod_{i = 1}^k \frac{n+1-i}{i}
\end{equation}
we can then plug in computed binomial coefficient ${{n}\choose{k}}$ in exact binomial p.d.f.

In [25]:
import functools
import operator

def binomial_coefficient(n, r):
    r = min(r, n - r)
    if r == 0: return 1
    numer = functools.reduce(operator.mul, range(n, n - r, -1))
    denom = functools.reduce(operator.mul, range(1, r + 1))
    return numer / denom

def binomial_pdf(n, k, p):
   return binomial_coefficient(n, k) * p**k * (1-p)**(n-k)

def binomial_cdf(n, k, p):
    k = n-k if 2*k>n else k # by symmetry
    result = 0
    for i in range(k+1):
        result += binomial_pdf(n, i, p)
    return result

print(binomial_cdf(400, 220, 0.5))

0.025520111564573993


---

# Trailing zeros
Q: Count trailing zeroes in factorial of an integer $N$

A: Consider the prime factorialization of $N$, e.g.

$$10! = 7 * 5^2 * 3^4 * 2^8$$

A trailing zero is always produced by prime factors of $5s$ and $2s$.
So answer is
$$\min(prime\_factors(N!, 5), prime\_factors(N!, 2))$$
Since $prime\_factors(N!, 2) > prime\_factors(N!, 5)$ (always), answer
is $prime\_factors(N!, 5)$

How to compute $prime\_factors(N!, 5)$?

\begin{equation}
Trailing\ 0s\ in\ N! = Count\ of\ 5s\ in\ prime\ factors\ of\ N!
                  = floor\left(\frac{N}{5}\right) + floor\left(\frac{N}{25}\right) + floor\left(\frac{N}{125}\right) + \ldots
\end{equation}

Example: how many trailing 0s in $100!$ ?

\begin{equation}
floor\left(\frac{100}{5}\right) + floor\left(\frac{100}{25}\right) + floor\left(\frac{100}{125}\right) + \ldots
 = 20 + 4 + 0 + \ldots
 = 24
\end{equation}

---
# Marbles in the bag, part I
Q: There is 4 marbles in a bag: 2 reds, 2 blacks. Randomly pick a marble, paint it
black if red, red if black. Find the expected number of times until all
the marbles have the same color.

A: Let $X$ be number of rounds till all marbles are the same color, and $M:N$ to state
in which we have $M$ marbles of one color and $N$ marble of the other. The goal is
to find $E[X|2:2]$

\begin{align}
E[N|2:2] & = &1 + E[N|3:1]\\
E[N|2:2] & = & 1 + (1 + \frac{3}{4} \times E[N|4:0] + \frac{1}{4} \times E[N|2:2])\\
E[N|2:2] & = & 1 + (1 + \frac{1}{4} \times E[N|2:2])\\
\frac{3}{4} \times E[N|2:2] & = & 2\\
E[N|2:2] & = & \frac{8}{3}\\
\end{align}

---
# Marbles in the bag, part II
Q: There is 4 marbles in a bag: 2 reds, 2 blacks. Randomly pick 2 marbles, paint the second
marble the color of the first marble. Find the expected number of times until all
the marbles have the same color.

A: Let $X$ be number of rounds till all marbles are the same color, $M:N$ to state
in which we have $M$ marbles of one color and $N$ marble of the other, and
$P(A, B|M:N)$ be the probability of drawing a marble of color $A$ then drawing
a marble of color $B$ if the initial state of the bag is $M:N$. The goal is
to find $E[X|2:2]$

\begin{align}
E[N|2:2] & = &1 + P(A, A|2:2) \times E[N|2:2] + P(A, B|2:2) \times E[N|3:1] \\
E[N|2:2] & = &1 + \frac{1}{3} \times E[N|2:2] + \frac{2}{3} \times E[N|3:1] \\
\frac{2}{3} \times E[N|2:2] & = &1 + \frac{2}{3} \times E[N|3:1] \\
\end{align}

Aside:

\begin{align}
E[N|3:1] & = &1 + P(A, A|3:1) \times E[N|3:1] + P(A, B|3:1) \times E[N|4:0] + P(B, A|3:1) \times E[N|2:2] \\
E[N|3:1] & = &1 + P(A, A|3:1) \times E[N|3:1] + P(B, A|3:1) \times E[N|2:2] \\
E[N|3:1] & = &1 + \frac{3}{4} \times \frac{2}{3} \times E[N|3:1] + \frac{1}{4} \times \frac{3}{3} \times E[N|2:2] \\
E[N|3:1] & = &1 + \frac{1}{2} \times E[N|3:1] + \frac{1}{4} \times E[N|2:2] \\
\frac{1}{2} \times E[N|3:1] & = &1 + \frac{1}{4} \times E[N|2:2] \\
E[N|3:1] & = &2 + \frac{1}{2} \times E[N|2:2] \\
\end{align}

Plug back in top equation:

\begin{align}
\frac{2}{3} \times E[N|2:2] & = &1 + \frac{2}{3} \times \left(2 + \frac{1}{2} \times E[N|2:2] \right) \\
\frac{2}{3} \times E[N|2:2] & = &1 + \left(\frac{4}{3} + \frac{1}{3} \times E[N|2:2] \right) \\
\frac{2}{3} \times E[N|2:2] & = &\frac{7}{3} + \frac{1}{3} \times E[N|2:2]\\
\frac{1}{3} \times E[N|2:2] & = &\frac{7}{3}\\
E[N|2:2] & = & 7\\
\end{align}

---
# Sum of normals
Q: let $X, Y \in N(0, 1)$ (standard normal distributions). What is the probability that $X > 5 \times Y$?

A: theorem: if $X \in N(\mu_x, \sigma_x)$, $Y \in N(\mu_y, \sigma_y)$ and $Z = X + Y$,
then $Z \in N(\mu_x + \mu_y, \sigma_x + \sigma_y)$

define $Y' = -5*Y$, then $P(X > 5 \times Y) = P(X - 5 \times Y > 0) = P(X + Y' > 0)$

according to theorem, $Z = X + Y' \in N(0, 6)$, hence $P(Z > 0) = \frac{1}{2}$

---
# The Bent Coin Lottery

_source: David Mackay's course on information theory, pattern recognition, and neural networks_

A biased coin with $p = 0.1$ will be tossed $N = 1000$ times. The outcome is
$X = x_1, x_2, \ldots, x_N$, e.g. $X = 0011010101000111000001101010110001010\ldots010110$

You can buy any of the $2^N$ possible tickets for \$1 each, before the coin-tossing.
If you own ticket $X$, you win \$1,000,000,000

Q1: To have a 99% chance of winning at the lowest possible cost, which tickets would you buy?

Q2: And how many tickets is that? express your answer in term of $2^?$

A: first, using binomial distributions find probabilities that the ticket has
$M$ 1's where $M = 0, 1, 2, \ldots, N$

In [5]:
Ps = [0.1 ** k * (1 - 0.1) ** (1000 - k) for k in range(1001)]

second, using combinatorics, find the number of ways $M$ heads and $N-M$ tails can be sampled

In [26]:
import math

Ts = [binomial_coefficient(1000, k) for k in range(1001)]

assert(sum(Ts) == 2 ** 1000)
assert(0.999999 < sum([p * t for (p, t) in zip(Ps, Ts)]) < 1.000001)

best_tickets = sorted(zip(Ps, Ts, range(1001)))

total_tickets = 0
P_winning = 0
fmt = 'buy {:.6e} tickets with {} heads in them - new probability: {:.6e}'
while P_winning < 0.99:
    (prob, num_tickets, num_heads) = best_tickets.pop()
    if P_winning + prob * num_tickets > 0.99:
        needed = math.ceil((0.99 - P_winning) / prob)
        total_tickets += needed
        P_winning += prob * needed
        print(fmt.format(needed, num_heads, P_winning))
        break
    else:
        total_tickets += num_tickets
        P_winning += prob * num_tickets
        print(fmt.format(num_tickets, num_heads, P_winning))

print('2 ^ {:.3f} tickets needed to be bought'.format(math.log(total_tickets, 2)))

buy 1.000000e+00 tickets with 0 heads in them - new probability: 1.747871e-46
buy 1.000000e+03 tickets with 1 heads in them - new probability: 1.959558e-44
buy 4.995000e+05 tickets with 2 heads in them - new probability: 1.097450e-42
buy 1.661670e+08 tickets with 3 heads in them - new probability: 4.093812e-41
buy 4.141712e+10 tickets with 4 heads in them - new probability: 1.144303e-39
buy 8.250291e+12 tickets with 5 heads in them - new probability: 2.556546e-38
buy 1.368173e+15 tickets with 6 heads in them - new probability: 4.755478e-37
buy 1.942806e+17 tickets with 7 heads in them - new probability: 7.575270e-36
buy 2.411508e+19 tickets with 8 heads in them - new probability: 1.054923e-34
buy 2.658018e+21 tickets with 9 heads in them - new probability: 1.304673e-33
buy 2.634096e+23 tickets with 10 heads in them - new probability: 1.450899e-32
buy 2.370686e+25 tickets with 11 heads in them - new probability: 1.465521e-31
buy 1.953840e+27 tickets with 12 heads in them - new probabili